## Convolutional Neural Networks
<img src="./Notes/Tut14.png">

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [24]:
# device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
# hyper-parameters
num_epochs = 10
batch_size = 4
learning_rate = 0.001

In [26]:
# dataset has PILImage images of range [0,1]
# we transform the to tensors of normalized range [-1,-1]
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            (0.5, 0.5, 0.5),
            (0.5, 0.5, 0.5)
        )
    ]
)

In [27]:
train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)
print(train_dataset)
print(test_dataset)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [28]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [29]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [30]:
# implement conv net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) 
        # 3 color channel, 6 output channel size, 5x5 kernel size
        self.pool = nn.MaxPool2d(2, 2)
        # 2x2 kernel size, stride 2
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [31]:
# the model
model = ConvNet().to(device)

In [32]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    params=model.parameters(),
    lr=learning_rate
)

In [33]:
# training loop
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        '''
        origin shape: [4, 3, 32, 32] = 4, 3, 1024
        input_layer:    3 input channels
                        6 output channels
                        5 kernel size
        '''
        images = images.to(device)
        labels = labels.to(device)

        # forward pass 
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward and optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 4000 == 0:
            print(f"Epoch: [{epoch+1}/{num_epochs}], Steps: [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}")
    print("")

Epoch: [1/10], Steps: [4000/12500], Loss: 2.3138
Epoch: [1/10], Steps: [8000/12500], Loss: 2.3024
Epoch: [1/10], Steps: [12000/12500], Loss: 2.2558

Epoch: [2/10], Steps: [4000/12500], Loss: 2.1246
Epoch: [2/10], Steps: [8000/12500], Loss: 1.7913
Epoch: [2/10], Steps: [12000/12500], Loss: 1.3108

Epoch: [3/10], Steps: [4000/12500], Loss: 1.6126
Epoch: [3/10], Steps: [8000/12500], Loss: 1.5885
Epoch: [3/10], Steps: [12000/12500], Loss: 1.6789

Epoch: [4/10], Steps: [4000/12500], Loss: 1.3668
Epoch: [4/10], Steps: [8000/12500], Loss: 1.9954
Epoch: [4/10], Steps: [12000/12500], Loss: 1.3118

Epoch: [5/10], Steps: [4000/12500], Loss: 0.9735
Epoch: [5/10], Steps: [8000/12500], Loss: 1.6273
Epoch: [5/10], Steps: [12000/12500], Loss: 1.1795

Epoch: [6/10], Steps: [4000/12500], Loss: 1.5386
Epoch: [6/10], Steps: [8000/12500], Loss: 1.5024
Epoch: [6/10], Steps: [12000/12500], Loss: 1.1892

Epoch: [7/10], Steps: [4000/12500], Loss: 1.6561
Epoch: [7/10], Steps: [8000/12500], Loss: 1.1251
Epoch: [

In [34]:
print("Finished Training")

Finished Training


In [35]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value, index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    acc = 100.0 * n_correct / n_samples
    print(f"Accuracy of the network: {acc}%")

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f"Acccuray of {classes[i]}: {acc}%")

Accuracy of the network: 57.94%
Acccuray of plane: 55.8%
Acccuray of car: 73.0%
Acccuray of bird: 40.4%
Acccuray of car: 29.8%
Acccuray of deer: 39.7%
Acccuray of dog: 56.0%
Acccuray of frog: 73.1%
Acccuray of horse: 62.6%
Acccuray of ship: 74.9%
Acccuray of truck: 74.1%
